In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import os
from sklearn.metrics import mean_squared_error
import random
import math
import time
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from random import choice
import pandas as pd
from sklearn.linear_model import Lasso
# from msvr import kernelmatrix
# from msvr import msvr

In [2]:
# class MSVR_F:
#     def __init__(self,ker  = 'rbf', C = 2,epsi = 0.001,par  = 0.8,tol  = 1e-10):
#         self.ker = ker
#         self.C = C
#         self.epsi = epsi
#         self.par = par
#         self.tol = tol
        
#     def fit(self,X_train,Y_train):
#         X_train = np.array(X_train)
#         Y_train = np.array(Y_train)
#         self.X_train = X_train
#         self.Beta = msvr(X_train, Y_train, self.ker, self.C, self.epsi, self.par, self.tol)
    
#     def predict(self,X_test):
#         X_test = np.array(X_test)
#         H = kernelmatrix('rbf', X_test, self.X_train, par);
#         return np.dot(H, self.Beta)

In [3]:
# Ecrire un générateur de blocs pour obtenir différents ensembles de données initiales
# data est la donnée, n est le nombre de parties en lesquelles elle est divisée, i est la première
# 写一个块生成器，用来获得不同的初始数据集
# data 是数据，n是分成几份，i是第几份

def getChunks(data,n,i):
    return data[ int(len(data)*(i)/n)+1 : (int(len(data)*(i+1)/n)-1)]
def getPourCentage(data,beginCentage,endCentage):
    return data[ (int(len(data)*beginCentage)): (int(len(data)*endCentage))]
#功能介绍：返回初始数据，返回全部训练数据，判断是否还有下一个batch，返回当前batch，
#Description de la fonction : retour des données initiales, retour de toutes les données d'entraînement, détermination de l'existence d'un lot suivant, retour du lot actuel.
class feeder_Ini_Train_Batch():
    def __init__(self,X,Y,beginCentage,endCentage,batch_size):
        self.X = X
        self.Y = Y
        #elf.X_train = getPourCentage(X,0,iniCentage)
        #elf.Y_train = getPourCentage(Y,0,iniCentage)
        self.X_test = getPourCentage(X,beginCentage,endCentage)
        self.Y_test = getPourCentage(Y,beginCentage,endCentage)
        self.batch_size = batch_size
        self.t = 1 # t C'est l'indexation et le temps. 就是索引和时间
    def getIni_X_Y(self,iniCentage):
        return getPourCentage(self.X,0,iniCentage),getPourCentage(self.Y,0,iniCentage)
    def getTrain_X_Y(self):
        return self.X_test,self.Y_test
    def hasThisBatch(self):
        if (self.t)*self.batch_size < len (self.X_test):
            return True
        else:
            return False
    def hasThisBatch_and_nextBath(self):
        if (self.t+1)*(self.batch_size) < len (self.X_test):
            return True
        else:
            return False
    def getThisBatch(self):
        if self.hasThisBatch() == True:
            actul_Batch_X = self.X_test[(self.t-1)*self.batch_size:(self.t)*self.batch_size]
            actul_Batch_Y = self.Y_test[(self.t-1)*self.batch_size:(self.t)*self.batch_size]
            return actul_Batch_X,actul_Batch_Y
        else:
            print('err index out')
    def getNextBatch_getThisBatch(self):
        if self.hasThisBatch() == True:
            next_Batch_X = self.X_test[(self.t)*self.batch_size:(self.t+1)*self.batch_size]
            next_Batch_Y = self.Y_test[(self.t)*self.batch_size:(self.t+1)*self.batch_size]
            return next_Batch_X,next_Batch_Y
        else:
            print('err index out')
    def goNext(self):
        self.t +=1



In [4]:
def getMeanSuperPourCentage(data,PourCentage): # Calculez la valeur moyenne du pourcentage supérieur des données. 计算排名前百分之多少的数据的均值
#     print(np.percentile(data, (PourCentage)))
    return np.mean([x for x in data if x >= np.percentile(data, (PourCentage))])
def getMeanSuperPourCentage_Martix_ParLine(data,PourCentage):
    return np.array([getMeanSuperPourCentage(x,PourCentage) for x in data ])

 # 0子模型 1编号时间 2权重 3class名 4俗称 5varepsilon加权错  6 Omega时间权重  7err普通RMSE错误  8 模型地址
 # 0 submodèle 1 temps de numérotation 2 poids 3 nom de classe 4 nom commun 5 erreur de pondération varepsilon 6 pondération temps oméga 7 erreur RMSE commune 8 adresse du modèle

def RandonSelectionModle(index,x,y , optitionInnErrOrCroissErr,numFlod = 3, indique_sousModle = 'Random_LS_LI_R_T'):  

    if indique_sousModle == 'Random_LI_R_T_KNN':
        indique_sousModle = choice(['Lasso','R_Forest','Tree','KNN'])
#     elif indique_sousModle == 'Random_LS_LI_R_T_KNN':
#         indique_sousModle = choice(['LSTM','Lasso','R_Forest','Tree','KNN'])
#     elif indique_sousModle == 'Random_LS_LI_R_T_SVR':
#         indique_sousModle = choice(['LSTM','Lasso','R_Forest','Tree','SVR'])
#     elif indique_sousModle == 'Random_LS_LI_R_T':
#         indique_sousModle = choice(['LSTM','Lasso','R_Forest','Tree'])
#     elif indique_sousModle == 'Random_LS_LI_R_KNN':
#         indique_sousModle = choice(['LSTM','Lasso','R_Forest','KNN'])
#     elif indique_sousModle == 'Random_LS_LI_T_KNN':
#         indique_sousModle = choice(['LSTM','Lasso','Tree','KNN'])
#     elif indique_sousModle == 'Random_LS_R_T_KNN':
#         indique_sousModle = choice(['LSTM','R_Forest','Tree','KNN'])
#     elif indique_sousModle == 'Random_LS_R_T':
#         indique_sousModle = choice(['LSTM','R_Forest','Tree'])
#     elif indique_sousModle == 'Random_LS_LI_T':
#         indique_sousModle = choice(['LSTM','Lasso','Tree'])
        
        print('Randomly selected indique_sousModle ',indique_sousModle)
    weight = 1 # weight of  this base model 投票权重
    varepsilon = 1 # Résultats des fonctions de perte 丢失函数结果 也就是 varepsilon 加权之后的错
    omega = 1 #  la pondération du temps 关于时间的权重
    RMSE = 1 # err 普通RMSE错误
    x_rnn = np.reshape(x, (x.shape[0], 1,x.shape[1]))
    kf = KFold(n_splits=numFlod)
    yhat_all_flod = []
    

    
#     elif indique_sousModle == 'LSTM' :
#         modele = Sequential()
#         modele.add(LSTM(500, input_shape =(1, x_rnn.shape[2]) , activation='relu'))
#         modele.add(Dense(500, activation='relu'))
#         modele.add(Dense(y.shape[1] , activation='sigmoid'))
#         # Compile model
#         modele.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#         modele.fit(x_rnn, y, epochs=6, batch_size=2000,  verbose=2)
#         if optitionInnErrOrCroissErr == 'CroissErr':
#             for train, test in kf.split(x_rnn):
#                 X_train_numFlod = np.array( [x_rnn[i] for i in train])
#                 Y_train_numFlod = np.array([y[i] for i in train])
#                 X_test_numFlod = np.array([x_rnn[i] for i in test])
#                 Y_test_numFlod = np.array([y[i] for i in test])
#                 modele_numFlod = Sequential()
#                 modele_numFlod.add(LSTM(500, input_shape =(1, x_rnn.shape[2]) , activation='relu'))
#                 modele_numFlod.add(Dense(500, activation='relu'))
#                 modele_numFlod.add(Dense(y.shape[1] , activation='sigmoid'))
#                 modele_numFlod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#                 modele_numFlod.fit(X_train_numFlod, Y_train_numFlod, epochs=4, batch_size=1000,  verbose=2)
#                 yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
#                 yhat_all_flod += yhat__numFlod.tolist()
#         return [modele,index,weight,modele.__class__,'LSTM',varepsilon,omega,RMSE,id(modele)] , yhat_all_flod
    
    
    
#     elif indique_sousModle == 'SVR' :
#         modele = MSVR_F()
#         modele.fit(x,y)
#         if optitionInnErrOrCroissErr == 'CroissErr':
#             for train, test in kf.split(x):
#                 X_train_numFlod = np.array( [x[i] for i in train])
#                 Y_train_numFlod = np.array([y[i] for i in train])
#                 X_test_numFlod = np.array([x[i] for i in test])
#                 Y_test_numFlod = np.array([y[i] for i in test])
#                 modele_numFlod = MSVR_F()
#                 modele_numFlod.fit(X_train_numFlod,Y_train_numFlod)
#                 yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
#                 yhat_all_flod += yhat__numFlod.tolist()        
#         return [modele,index,weight,modele.__class__,'SVR',varepsilon,omega,RMSE,id(modele)],yhat_all_flod
    

    if indique_sousModle == 'KNN':
        modele = KNeighborsRegressor(n_neighbors=15)
        modele.fit(x,y)
        if optitionInnErrOrCroissErr == 'CroissErr':
            for train, test in kf.split(x):
                X_train_numFlod = np.array( [x[i] for i in train])
                Y_train_numFlod = np.array([y[i] for i in train])
                X_test_numFlod = np.array([x[i] for i in test])
                Y_test_numFlod = np.array([y[i] for i in test])
                modele_numFlod = KNeighborsRegressor(n_neighbors=15)
                modele_numFlod.fit(X_train_numFlod,Y_train_numFlod)
                yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
                yhat_all_flod += yhat__numFlod.tolist()        
        return [modele,index,weight,modele.__class__,'KNeighborsRegressor',varepsilon,omega,RMSE,id(modele)],yhat_all_flod
    
    elif indique_sousModle == 'R_Forest':
        modele = RandomForestRegressor(n_estimators=10, )
        modele.fit(x,y)
        if optitionInnErrOrCroissErr == 'CroissErr':
            for train, test in kf.split(x):
                X_train_numFlod = np.array( [x[i] for i in train])
                Y_train_numFlod = np.array([y[i] for i in train])
                X_test_numFlod = np.array([x[i] for i in test])
                Y_test_numFlod = np.array([y[i] for i in test])
                modele_numFlod = RandomForestRegressor(n_estimators=10,)
                modele_numFlod.fit(X_train_numFlod,Y_train_numFlod)
                yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
                yhat_all_flod += yhat__numFlod.tolist()
        return [modele,index,weight,modele.__class__,'RandomForestRegressor',varepsilon,omega,RMSE,id(modele)],yhat_all_flod
            
    elif indique_sousModle == 'Lasso':
        modele = Lasso(alpha=0.1)
        modele.fit(x,y)
        if optitionInnErrOrCroissErr == 'CroissErr':
            for train, test in kf.split(x):
                X_train_numFlod = np.array( [x[i] for i in train])
                Y_train_numFlod = np.array([y[i] for i in train])
                X_test_numFlod = np.array([x[i] for i in test])
                Y_test_numFlod = np.array([y[i] for i in test])
                modele_numFlod = Lasso(alpha=0.1)
                modele_numFlod.fit(X_train_numFlod,Y_train_numFlod)
                yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
                yhat_all_flod += yhat__numFlod.tolist()        
        return [modele,index,weight,modele.__class__,'LassoRegression',varepsilon,omega,RMSE,id(modele)],yhat_all_flod
        
    elif indique_sousModle == 'Tree':
        modele = DecisionTreeRegressor()
        modele.fit(x,y)
        if optitionInnErrOrCroissErr == 'CroissErr':
            for train, test in kf.split(x):
                X_train_numFlod = np.array( [x[i] for i in train])
                Y_train_numFlod = np.array([y[i] for i in train])
                X_test_numFlod = np.array([x[i] for i in test])
                Y_test_numFlod = np.array([y[i] for i in test])
                modele_numFlod = DecisionTreeRegressor()
                modele_numFlod.fit(X_train_numFlod,Y_train_numFlod)
                yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
                yhat_all_flod += yhat__numFlod.tolist()        
        return [modele,index,weight,modele.__class__,'DecisionTreeRegressor',varepsilon,omega,RMSE,id(modele)],yhat_all_flod
        
def makePredictionModele(modele,x):
    if type(modele) in [type(RandomForestRegressor()), 
                          type(KNeighborsRegressor()),
                          type(Lasso()),
                          type(DecisionTreeRegressor())] :
#                                   type(MSVR_F())

        return  modele.predict(x)
    else:
        print('报错，不在列表中 Error reported, not in the list')

def beta_fonction(k,t,a,b):  
    return 1/(1+ math.exp(-a*(t-k-b))) 

def vote_mean(dic_expert, actul_Batch_X, actul_Batch_Y, afficher_detail = False):
    H_res = np.array( [[0.0]* len(actul_Batch_Y[0])] *len(actul_Batch_Y))
    print(H_res)
    sumWeigt = 0
    
    list_RMSE_SousModele = []

    for key,value in dic_expert.items():
        yhat_sousM = makePredictionModele(value[0],actul_Batch_X)
        
        if np.array(yhat_sousM).ndim == 1:
            print('type([[z] for z in yhat_sousM])  ',type([[z] for z in yhat_sousM]))
            yhat_sousModel = np.array([[z] for z in yhat_sousM])
        else:
            yhat_sousModel = yhat_sousM
            print(type(yhat_sousModel))
        print('Output of each sub-model',yhat_sousModel)
        H_res =H_res + yhat_sousModel*value[2]
        sumWeigt += value[2] #  2 représente les poids des votes des sous-modèles 2号代表子模型投票权
        list_RMSE_SousModele.append(aRMSE( actul_Batch_Y ,yhat_sousModel))
    yhat_H = np.array(H_res)/sumWeigt # L'obtention des résultats prédits pour H
    
#     print('投票结束之后的输出', yhat_H)
#     print('Y的原始值', actul_Batch_Y)
    if afficher_detail == True:
        print( 'list_RMSE_SousModele ', list_RMSE_SousModele, 'mean of list_RMSE_SousModele', np.mean(list_RMSE_SousModele) )
        print(' RMSE resultat vote', aRMSE( actul_Batch_Y ,yhat_H))
    return yhat_H


def vote_OnlyMaxERRWeight(dic_expert, actul_Batch_X, actul_Batch_Y):
    the_key = 0
    maxWeight = 0
    for key,value in dic_expert.items():
        if dic_expert[key][5] > maxWeight:
            maxWeight = dic_expert[key][5]
            the_key = key
    yhat_H = makePredictionModele(dic_expert[the_key][0],actul_Batch_X)
    err_H =  aRMSE(actul_Batch_Y,yhat_H)# err as RMSE 
    print('Sub-model selected to vote, no. ', the_key, ' 其 RMSE', err_H)
    return yhat_H

def vote_OnlyMaxESpWeight(dic_expert, actul_Batch_X, actul_Batch_Y):
    the_key = 0
    maxWeight = 0
    for key,value in dic_expert.items():
        if dic_expert[key][2] > maxWeight:
            maxWeight = dic_expert[key][2]
            the_key = key
    yhat_H = makePredictionModele(dic_expert[the_key][0],actul_Batch_X)
    err_H =  aRMSE(actul_Batch_Y,yhat_H)# err as RMSE 
    print('Sub-model selected to vote, no. ', the_key, ' with its RMSE', err_H)
    return yhat_H

def updatingALLSousModele(dic_expert, x, y):
    print (' Update sub-model program start' )
    x_rnn = np.reshape(x, (x.shape[0], 1,x.shape[1]))
    for key,value in dic_expert.items():
        if type(value[0]) in [type(Sequential())]:
            value[0].fit(x_rnn, actul_Batch_Y)

            
def get_filename(path,filetype):  # 输入路径、文件类型例如'.csv'
    name = []
    for root,dirs,files in os.walk(path):
        for i in files:
            if os.path.splitext(i)[1]==filetype:
                name.append(i)    
    return name

def aRMSE(y_true,y_pred):
    return np.mean(mean_squared_error(y_true, y_pred, squared=False, multioutput='raw_values'))

In [5]:


path_X_Y_data = 'simulationDonnes/'
# Y = np.load(path_X_Y_data + 'Y_CU_BEMS_19_normale_nettoy.npy' ,allow_pickle=True)
# X = np.load(path_X_Y_data + 'X_CU_BEMS_19_normale_nettoybackTimePoint3PCA10.npy' ,allow_pickle=True)

Y = np.load(path_X_Y_data + 'Y(2ans)Norma_futureTimePoint0.npy' ,allow_pickle=True)
X = np.load(path_X_Y_data + 'X(2ans)Norma_backTimePoint5.npy' ,allow_pickle=True)

        
while(True):
    
    dic_expert = {} #Conserver un dictionnaire des sous-modèles
    timeList, errList, innErrList, testErrNextBatchList,nFlodErr= [],[],[],[],[] #Liste des temps, liste de la précision 
    Resulta_list, Ture_list = [],[] #Recorded information: predicted results and true values
    dic_expert = {} #Conserver un dictionnaire des sous-modèles
    ini_pourCentage = 0.01 # Le rapport initial doit être défini 
    testStar,testEnd = 0.2,0.99
    HowManyDaysForBatch =14
    batch_size = 1400*HowManyDaysForBatch# Taille du bloc
    a,b = 0.35,1
    time_Ini = 0 # Le temps nécessaire à l'initialisation est enregistré ici
    varepsilon_extreme_bord = 1.2 # Coefficient permettant de vérifier si le sous-modèle fonctionne correctement
    maxNumSousModle = 10 # Nombre maximal de sous-modèles
    testBatchNum = 100 #Contrôler la quantité de circulation pendant l'expérience
    Q3 = 75 # Définir les coefficients pour le calcul des poids d'instance
    optionPCA, PCA_size = True, 25
#     optitionInnErrOrCroissErr = 'InnErr'
    optitionInnErrOrCroissErr = 'CroissErr'
    optitionAddSousModele = True
    optionVote ='Mean'
    # optionVote ='OnlyMaxERRWeight'
    # optionVote = 'vote_OnlyMaxESpWeight'
    indicateur_sousModel = 'Random_LI_R_T_KNN' #['Random','LSTM','Lasso','KNN','R_Forest'] # 0 随机，1 LSTM，
    cStepAugment = 1.4
    updateOrNon = False


    if optionPCA == True:
        pca = PCA(n_components=PCA_size)
        principalComponents = pca.fit_transform(X)
        pca.explained_variance_ratio_
        importance = pca.explained_variance_ratio_
        plt.scatter(range(1,PCA_size+1),importance)
        plt.plot(range(1,PCA_size+1),importance)
        plt.title('Scree Plot')
        plt.xlabel('Factors')
        plt.ylabel('Eigenvalue')
        plt.grid()
        plt.show()
        X = pca.fit_transform(X)
    
    myFeeder = feeder_Ini_Train_Batch(X,Y,testStar,testEnd,batch_size) # X,Y beginCentage,endCentage,batch_size 
    X_train,Y_train = myFeeder.getIni_X_Y(ini_pourCentage) #这里才设定初始比例
    X_test,Y_test = myFeeder.getTrain_X_Y()

    iniFlage = True

    while ((testBatchNum > myFeeder.t)  and myFeeder.hasThisBatch()):

        # Créer un sous-modèle initialisé, code 0
        start_Ini = time.process_time()# Heure de début d'enregistrement记录开始时间
        if iniFlage == True:
            Delta = [1/ myFeeder.batch_size]* myFeeder.batch_size #Valeur par défaut de delta delta的默认值 
            dic_expert[0], err_numFlod= RandonSelectionModle(0,X_train, Y_train,optitionInnErrOrCroissErr, indique_sousModle = indicateur_sousModel)
            iniFlage = False
        time_Ini = round(time.process_time()-start_Ini,5) # Le temps nécessaire à l'initialisation est enregistré ici 这里记录了初始化需要的时间
        start = time.process_time()# Heure de début d'enregistrement 记录开始时
        
        #----------------VOTE------------------------------------------------------
        actul_Batch_X, actul_Batch_Y  =  myFeeder.getThisBatch()
        # Calculer la performance de Ht-1 à t 
        if optionVote == 'Mean':
            yhat_H = vote_mean(dic_expert,actul_Batch_X, actul_Batch_Y,afficher_detail=True)
        elif optionVote == 'Max':
            yhat_H = vote_OnlyMaxWeight(dic_expert,actul_Batch_X, actul_Batch_Y)

        print('vote is over')
        #----------------------------------------------------------------------

        #----------------Confirm instance weights and record current cluster performance------------------------------------------------------
        ERR_absolu_H = np.abs(actul_Batch_Y - yhat_H) #ERR_absolu (martix)
        Resulta_list.append(yhat_H)
        Ture_list.append(actul_Batch_Y)
        err_H = aRMSE(actul_Batch_Y,yhat_H)# err as RMSE 
        meanERR_absolu_H = np.mean(ERR_absolu_H, axis=1)
    #     Delta = getMeanSuperPourCentage_Martix_ParLine(ERR_absolu_H,Q3)
    #     varepsilon_H = np.average(meanERR_absolu_H,weights=Delta)

        varepsilon_H = np.average(meanERR_absolu_H)
        errList.append(err_H) 
        print('Instance weights, recording current group performance, over')
        #----------------确认实例权重，记录当前群性能------------------------------------------------------

        #----------------Alarm in case of abnormality------------------------------------------------------
        if err_H >10:
            print('time: ',myFeeder.t,'Une erreur majeure s est produite：',err_H) 
        #----------------当群性能异常时，报警------------------------------------------------------


        #------------------Add a new sub-model and confirm its weights (from cross-validation)-----------------------------
        # 添加一个新的子模型    
        # 这里有新改动，未在     meanERR_absolu < 0.1
        bord = 1 
        while(optitionAddSousModele):
            print('Start adding new sub-models')
            dic_expert[myFeeder.t], yhat_new_numFlod = RandonSelectionModle(myFeeder.t, actul_Batch_X, actul_Batch_Y, optitionInnErrOrCroissErr,indique_sousModle = indicateur_sousModel)

        
            if optitionInnErrOrCroissErr == 'CroissErr':
                ERR_absolu_new_numFlod = np.abs(actul_Batch_Y - yhat_new_numFlod) 
                meanERR_absolu_new_numFlod = np.mean(ERR_absolu_new_numFlod, axis=1)
                err_new_numFlod = aRMSE(actul_Batch_Y,yhat_new_numFlod) 
                varepsilon_new = np.average(meanERR_absolu_new_numFlod,weights=Delta)
                if varepsilon_new < varepsilon_H*bord:
                    nFlodErr.append(err_new_numFlod)
                    break
                else:
                    bord = bord*cStepAugment
                
            elif optitionInnErrOrCroissErr == 'InnErr':

                yhat_new_inner = makePredictionModele(dic_expert[myFeeder.t][0], actul_Batch_X)
                ERR_absolu_new_inner = np.abs(actul_Batch_Y - yhat_new_inner) 
                meanERR_absolu_new_inner = np.mean(ERR_absolu_new_inner, axis=1)
                err_new_inner = aRMSE(actul_Batch_Y,yhat_new_inner) 
                varepsilon_new = np.average(meanERR_absolu_new_inner,weights=Delta)
                break
        #------------------添加一个新的子模型 ，确认其权重（来自交叉验证）-----------------------------
        
        if optitionAddSousModele:
            #------------------Recording training errors for new sub-models-----------------------------
            yhat_new_inner = makePredictionModele(dic_expert[myFeeder.t][0], actul_Batch_X)
            err_new_inner = aRMSE(actul_Batch_Y,yhat_new_inner)         
            innErrList.append(err_new_inner)
            #------------------对新的子模型记录训练错误-----------------------------

            #------------------Test error of the model generated by the previous block in the next block-----------------------------
            if myFeeder.hasThisBatch_and_nextBath():
                next_batch_test_X,  next_batch_test_Y =  myFeeder.getNextBatch_getThisBatch()
                yhat_next_batch_test = makePredictionModele(dic_expert[myFeeder.t][0], next_batch_test_X)
                err_test_NextBatch = aRMSE(next_batch_test_Y, yhat_next_batch_test)         
                testErrNextBatchList.append(err_test_NextBatch)
            #------------------对上一个块的子模型在下一块的测试错误-----------------------------


        #------------------Calculate the performance of the old sub-model based on the weighted strengths in the current block-----------
        Varepsilon_list = [] #Collecte de la liste des pesées séparées de Varepsilon pour faciliter la recherche des valeurs max-min. 收集Varepsilon单独称list，方便找最大最小值
        for key,value in dic_expert.items():
            yhat_oldModel = makePredictionModele(value[0], actul_Batch_X)
            ERR_absolu_oldModel = np.abs(actul_Batch_Y - yhat_oldModel)
            mean_ERR_absolu_oldModel = np.mean(ERR_absolu_oldModel, axis=1)
            err_oldModel = aRMSE(actul_Batch_Y,yhat_oldModel) # RMSE
            varepsilon_oldModel = np.average(mean_ERR_absolu_oldModel,weights=Delta)
    #         if varepsilon_oldModel > varepsilon_H*varepsilon_extreme_bord: 
    #             value[5] = varepsilon_H*varepsilon_extreme_bord
    #         else:
    #             value[5] = varepsilon_oldModel
            value[5] = varepsilon_oldModel
            Varepsilon_list.append(value[5])
            value[7] = err_oldModel # enregrister RMSE
    #         for key,value in dic_expert.items():   #normalisation 
    #             value[5] = (value[5] - min(Varepsilon_list))/(max(Varepsilon_list) - min(Varepsilon_list))
    #     for key,value in dic_expert.items(): 
    #         print('key ', key, ' value[5] ', value[5] )
        #------------------根据当前块中被加权的实力，计算旧的子模型的表现（7号普通错误），并更新权重（5号根据加权错误）-----------


        #------------------Calculating time weights ----------
        for key,value in dic_expert.items(): 
            numerator = beta_fonction(value[1],myFeeder.t,a,b)
            denominator = 0
            for j in range(0, myFeeder.t - value[1] +1): # 
                denominator +=  beta_fonction(myFeeder.t - j,myFeeder.t,a,b)
#             print('denominator  ' , denominator)
#             if  denominator == 0:
#                 print(' myFeeder.t  ', myFeeder.t,' value[1]', value[1] )
#                 for j in range(0, myFeeder.t - value[1] +1): # 
#                     print('j :', j , ' beta_fonction(myFeeder.t - j,myFeeder.t,a,b)', beta_fonction(myFeeder.t - j,myFeeder.t,a,b))
            value[6] =  numerator/denominator
        for key,value in dic_expert.items(): 
            print('key ', key, ' value[6] ', value[6] )
        #------------------计算6 Omega, 也就是时间权重 omega越小，越重要-----------


        #------------------Calculating sub-model weights-----------
        # Calculer les poids des sous-modèles
        weight_list = []
        for key,value in dic_expert.items(): 
            denominator = 0 
            for j in range(0, myFeeder.t - value[1] +1): #  from 0 to  t-k
                if ((myFeeder.t-j) in dic_expert.keys()):
                    denominator += dic_expert[myFeeder.t-j][6]*dic_expert[myFeeder.t-j][5]+0.1  # Omega 权重 * 5 当前块被加权的错误    
            value[2] = math.log(1/denominator)  
            weight_list.append(math.log(1/denominator)) 
        for key,value in dic_expert.items(): 
            value[2] = (value[2] - min(weight_list)+0.1)/(max(weight_list) - min(weight_list)+0.1)
        for key,value in dic_expert.items(): 
            print('key ', key, 'value[2] ',value[2] )
        #------------------Calculating sub-model weights-----------


        #------------------Removing the worst model -----------
        if len(dic_expert) > maxNumSousModle:
            maxErr = 0
            del_key = 0
            for key,value in dic_expert.items(): 
                if dic_expert[key][7] > maxErr:
                    maxErr = dic_expert[key][7]
                    del_key = key
            dic_expert.pop(del_key)
        #------------------去掉模型 最差策略 Sous-modèle de suppression de la pire stratégie-----------
        
        
        #------------------Whether the sub-model LSTM is updated-----------
        if updateOrNon:
            updatingALLSousModele(dic_expert,actul_Batch_X,actul_Batch_Y) 
        #------------------子模型LSTM是否更新-----------


        # 记录时间 Durée d'enregistrement
        timeList.append(round(time.process_time()-start,3)) #记录时间

        print('Average errors so far np.mean(errList)', np.mean(errList))    #展示平均
        print('Current Error err_H', err_H)

        #进入下一块
        print('myFeeder.t ', myFeeder.t)
        myFeeder.goNext()

        # print(errList)

    
    patch = "resulta/"
    # nameFileSave = "CUBEM_resulta"
    nameFileSave = "UKDALE_resulta"
    res_oneTour = {}
    res_oneTour['Resulta_list'] =  Resulta_list
    res_oneTour['Ture_list'] =  Ture_list
    np.save(patch + nameFileSave + '.npy', res_oneTour , allow_pickle=True, fix_imports=True)

    break;
